# Instructions
The following code was designed in order to track the location of a single animal across the course of a single video session.  After initally loading in the video, the user is able to crop the video frame using a rectangle selection tool.  A background reference frame is then specified, either by taking a median of several frames in the video, or by the user providing a short video of the same environment without an animal in it.  By comparing each frame in the video to the reference frame, the location of the animal can be tracked.  It is imperative that the reference frame of the video is not shifted from the actual video.  For this reason, if a separate video is supplied, it is best that it be acquired on the same day as the behavioral recording.  The animal's center of mass, in x,y coordinates, is then recorded, as well as the distance in pixels that the animal moves from one frame to the next. Lastly, the user can specify regions of interest in the frame (e.g. left, right) using a polygon drawing tool and record for each frame whether the animal is in the region of interest.  Options for summarizing the data are also provided. 

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

---
# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import LocationTracking_Functions as lt
import holoviews as hv

---
# 2. User Sets Directory and File Information, and Specifies ROI Names, if any.
Below the user sets the directory file path (path of the folder where the video file is), the file name of the video to be analyzed, the frame on which the analysis is to begin (0 is the first frame), and the last frame to be analyzed (set `'end' : None` if processing entire video).  Additionally, the user defines the names of any regions of interest.  Any number of regions of interst can be used.  If no regions of interest are to be used, set `region_names = None`. By modifying 'stretch', one can alter the relative width/height of the presented output.

***Windows Users:*** Place an 'r' in front directory path (e.g. r"zp\Videos") to avoid mishandling of forward slashes.

In [ ]:
video_dict = {
    'dpath' : "/Users/me/Videos/",  
    'file' : 'video.mpg',
    'start' : 0, 
    'end' : None
}
stretch = dict(width=1, height=1)

region_names = None #['Left','Right']

---
# 3. Load Video and Crop Frame if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.

If the size of the image is too small/large, alter the first line of code.  100 is the standard size.  200 will produce an image 2x the size, and so on.

In [ ]:
%%output size=100

image,crop,video_dict = lt.LoadAndCrop(video_dict,stretch,cropmethod='Box')
image

---
# 4. Define Reference Frame for Location Tracking.
For location tracking to work, view of box without animal must be provided.  Below there are two ways to do this.  **Option 1** provides a method to remove the animal from the video.  This option works well provided the animal doesn't stay in the same location for >50% of the session. Alternatively, with **Option 2**, the user can simply define a video file in the same folder that doesn't have in animal in it.  Option 1 is generally recormmended. 

### Option 1 - Create reference frame by removing animal from video
The following code takes 100 random frames across the session and creates an average of them by taking the median for each pixel.  This will remove influence of animal on any given frame.

In [ ]:
%%output size=100

reference, image = lt.Reference(video_dict,stretch,crop=crop,num_frames=100) 
image

### Option 2 - User specifies video of empty box
The following code allows the user to specify a different file.  Set video_dict['altfile'] to the alternative filename.  Notably, an average is still taken of multiple frames.

In [ ]:
%%output size=100

video_dict['altfile'] = 'EmptyBox.avi' 
reference, image = lt.Reference(video_dict,stretch,crop=crop,num_frames=100,altfile=True) 
image

# 5. (Optional) Use Interactive Plot to Define Regions of Interest.  

After running cell below, draw regions of interest on presented image in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.

***Note*** that there are no problems if regions overlap.

In [ ]:
%%output size=100 

plot,poly_stream = lt.ROI_plot(reference,region_names,stretch)
plot

---
# 6. (Optional) Define Scale for Distance Calculations

### 6a. Select two points of known distance

After running cell below, click on any two points and the distance between them, in pixel units, will be presented/returned. Will be used to convert pixel distance to other scale. Note that once drawn, points can be dragged or you can click again.

In [ ]:
%%output size = 100

dist_plot, dist = lt.DistanceTool(reference,stretch)
dist_plot

### 6b. Define real-world distance between points
Below, set the distance between the points selected below, and the scale. Note that scale can be any desired text.

In [ ]:
scale_dict = {
    'distance' : 100,
    'scale' : 'cm' 
}

---
# 7. Track Location

### 7a. Set Location Tracking Parameters
Location tracking examines the deviance of each frame in a video from the reference frame on a pixel by pixel basis.  For each frame it calculates the center of mass for these differences (COM) to define the center of the animal.  

In order to improve accuracy, the parameter loc_thresh is used to remove the influence of pixels that are minimally different from the reference frame.  For each frame relative to the reference frame, the distribution of absolute difference values across pixels is examined and only values above loc_thresh percentile are considered.  We have been using 99-99.5 and this works well.  Values can range from 0-100 and floating point numbers are accepted.

The parameters: use_window, window_size, and window_weight are employed to reduce the chance that any objects other than the animal  that might enter the frame (e.g. the hand of the experimenter) influence location tracking.  For each frame, a square window with the animal's position on the prior frame at its center is given more weight when searching for it's location (because an animal presumably can't move far in a fraction of a second).  When window_weight is set to 1, pixels outside of the window are not considered at all; at 0, they are given equal weight.  Notably, setting a high value that is still not equal to 1 (e.g. 0.9) should allow the program to more rapidly find the animal if by chance it moves out of the window.

The paramater method accepts values of 'abs', 'light', and 'dark'. 'abs' does not take into consideration whether the animal is lighter or darker than the background and will therefore track the animal across a wide range of backgrounds. 'light' assumes the animal is lighter than the background, and 'dark' assume the animal is darker than the background. 'abs' generally works well, but there are situations in which you may wish to use the others.

In [ ]:
tracking_params = {
    'loc_thresh' : 99, 
    'use_window' : True, 
    'window_size' : 100, 
    'window_weight' : .9, 
    'method' : 'abs' 
}

### 7b. Display Examples of Location Tracking to Confirm Threshold
In order to confirm threshold is working, a subset of images is analyzed and displayed using the selected tracking_params.  The original image is displayed on top and the difference values are presented below.  The center of mass (COM) is pinpointed on images.  Notably, because individual frames are used, window settings are not applicable here.  Because of this, actual tracking in video is likely to be better.

The user can change the number examples below as they see fit.

In [ ]:
%%output size = 100
%%time

examples = 5 
images=lt.LocationThresh_View(video_dict,reference,tracking_params,crop=crop,examples=examples,stretch=stretch)
images.cols(2)

### 7c. Track Location and Save Results to .csv File
For each frame the location of the animal's center of mass is recorded in x/y coordinates.  If ROIs are supplied, for each frame it is determined whether the animal is in each of the ROIs.  Frame-by-frame distance is also calculated in pixel units.  This data is returned in a Pandas dataframe with columns: frame, x, y, dist, and whether the animal is in each ROI specified (True/False).  Data is saved to a .csv in the same folder as the video.  First 5 rows of data are presented.

In [ ]:
location=lt.TrackLocation(video_dict,tracking_params,reference,crop=crop)
if region_names != None: 
    location = lt.ROI_Location(reference,location,region_names,poly_stream) 
if 'scale_dict' in locals():
    location = lt.ScaleDistance(scale_dict, dist, df=location, column='Distance_px')
    
location.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_LocationOutput.csv')
location.head()

### 7d. Display Animal Distance/Location Across Session
Below, the animals distance and location across the video is plotted.  Smooth traces are expected in the case where the animal is tracked consistently across the session.  Under heatmap, sigma controls 'binning' of location. When 'sigma = None' default value is provided; but sigma can also be set to any value.

In [ ]:
%%output size=100

#Plot Distance Across Session
w, h = 600,200 #specify width and height of plot
dist_plot = hv.Curve((location['Frame'],location['Distance_px']),'Frame','Pixel Distance').opts(
    height=h,width=w,color='red',title="Distance Across Session",toolbar="below")

#Plot Trace of Animal Across Session and Generate Heatmap
tracks = lt.showtrace(reference,location,color="red",alpha=.05,size=2,stretch=stretch)
heatmap = lt.Heatmap(reference, location, sigma=None,stretch=stretch)
(tracks+heatmap+dist_plot).cols(1)

---
# 8. (Optional) Create Binned Summary Report and Save
The code below allows the user to either save a csv containing summary data for user-defined bins (e.g. proportion of time in each region and distance travelled for each minute) or a session-wide average. 

***If you only want a session avg***, set `bin_dict = None`

***If you are not using ROIs***, in the code below, set value of 'region_names' within function to None: `region_names=None`.  Otherwise, keep `region_names=region_names`

To specify bins, set bin_dict using the following notation, where start and stop represent time ***in frames***:

```
bin_dict = {
    'BinName1': (start, stop),
    'BinName2': (start, stop),
    'BinName3': (start, stop),
}

```

Wanna get fancy?  Check out how to use Python dictionary comprehensions to make 20, 30 second bins, using 1 line of code:

`bin_dict = {x:(x*30*60,(x+1)*30*60) for x in range(20)}`


In [ ]:
bin_dict = {
    '1':(0,10),
    '2':(10,20),
    '3':(20,30)
}

summary = lt.Summarize_Location(location, video_dict, bin_dict=bin_dict, region_names=region_names)
summary = lt.ScaleDistance(scale_dict,dist,df=summary,column='Distance_px') if 'scale_dict' in locals() else summary
summary.to_csv(os.path.splitext(video_dict['fpath'])[0] + '_SummaryStats.csv')
summary.head()

---
# 9. (Optional) View Video of Tracking
**Note** that tracking must be done before this (Step 7c). 
The code below allows the user to play back a section of the analyzed video, with the center of mass of the animal marked, to confirm tracking.  User defines the start and end frames of the section they would like to watch.  Of note, if the user sets the start frame for location tracking to something other than 0 in Step 2, the frame start and end here will be relative to that. If the displayed size of the video should be changed, the user can set the `resize` key below to a tuple with the desired width and height, in pixel units... for example: `'resize' : (width,height)`.  The specified width and height should be integers. Lastly, the video can be saved by setting `'save_video' : True`. `fps` controls speed at which frames are presented.

In [ ]:
%%output size=10
display_dict = {
    'start' : 0, 
    'stop' : 30, 
    'fps' : 30,
    'resize' : None,
    'save_video' : False
}

lt.PlayVideo(video_dict,display_dict,location,crop=crop)